In [45]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim
from sklearn.feature_extraction.text import TfidfVectorizer
import codecs
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords

a = set(stopwords.words("english")) |set(stopwords.words("french")) ## union of english and french stop_words


# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot 

[nltk_data] Downloading package punkt to /home/maza/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/maza/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [46]:
good_format=[] ## stock text that can be open with "utf-8" encoding
nb_document=33226
for i in range(nb_document):
    try :
        r_emplacement="/home/maza/Workspace/INF554/kaggle/node_information/text/"+str(i)+".txt"

        document=codecs.open(r_emplacement,"r",encoding="utf-8")

        name=str(i)+"_important.txt" ##renamed the file "formername+important.txt"
        w_emplacement="/home/maza/Workspace/INF554/kaggle/node_information/without_noise/"+name
        text_file = codecs.open(w_emplacement, "w",encoding="utf-8")
        noise=["Pour paramétrer :","cookies","COOKIES","droit d'auteur","Newsletter","newsletter"] ## We remove this noise
        noise_bool=False
        with open(r_emplacement,"r",encoding="utf-8") as f:
            cmpt=0
            for line in f :
                #print(line)
                if not line.strip():continue 
                if(cmpt>10):
                    for n in noise :
                        if(n in line):

                            noise_bool=True
                            break ;
                if(noise_bool==True):
                    break;
                text_file.write(line)
                cmpt+=1
        text_file.close()        

    except:
        good_format.append(i)
        

In [47]:
fail=set([x for x in np.arange(nb_document)]).difference(set(good_format)) ## id of doxuments wich are not in good_format

In [48]:
## We will open on fail_document with encoding="iso-8859-1"
for i in fail:
        r_emplacement="/home/maza/Workspace/INF554/kaggle/node_information/text/"+str(i)+".txt"

        document=codecs.open(r_emplacement,"r",encoding="iso-8859-1")
        name=str(i)+"_important.txt" ##renamed the file "formername+important.txt"
        w_emplacement="/home/maza/Workspace/INF554/kaggle/node_information/without_noise/"+name
        text_file = codecs.open(w_emplacement, "w",encoding="iso-8859-1")
        noise=["Pour paramétrer :","cookies","COOKIES","droit d'auteur","Newsletter","newsletter"]
        noise_bool=False
        with open(r_emplacement,"r",encoding="iso-8859-1") as f:
            cmpt=0
            for line in f :
                #print(line)
                if not line.strip():continue 
                if(cmpt>10):
                    for n in noise :
                        if(n in line):

                            noise_bool=True
                            break ;
                if(noise_bool==True):
                    break;
                text_file.write(line)
                cmpt+=1
        text_file.close()        



In [54]:
def sent_to_words(sentences): ## We 
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) ## Removes punctuations

data_words = list(sent_to_words(data))



In [55]:
import json 
with open("data_words.json","w") as outfile:
    json.dump(data_words,outfile) ## We save data_words

In [56]:
a=list(a) ## Reminder that a is a set stop_words
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum required occurences of a word 
                             stop_words=frozenset(a),          # remove stop words
                             lowercase=False,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                                       
                            )

In [57]:
data_vectorized = vectorizer.fit_transform(data1)

/home/maza/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['taient', 'tais', 'tait', 'tant', 'tante', 'tantes', 'tants', 'tiez', 'tions'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [58]:
import pickle

pickle.dump(data_vectorized,open("data_vectorized.pkl","wb")) ##We save data_words data_vectorized

In [59]:
import pickle
data_vectorized=pickle.load(open("data_vectorized.pkl","rb"))

In [60]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_components =20,# Number of topics

                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=20, n_jobs=None,
                          perp_tol=0.1, random_state=None,
                          topic_word_prior=None, total_samples=1000000.0,
                          verbose=0)


In [61]:
import sklearn

print("Log Likelihood: ", lda_model.score(data_vectorized))


print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -404779938.4301665
Perplexity:  6751.2744299216765
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'batch',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 20,
 'n_jobs': None,
 'perp_tol': 0.1,
 'random_state': None,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [62]:
# Create Document - Topic Matrix
nb_document=33226
lda_output = lda_model.transform(data_vectorized)

# column names
topicnames = ["Topic" + str(i) for i in range(20)]

# index names
docnames = ["Doc" + str(i) for i in range(nb_document)]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic



In [63]:
# Styling
def color_green(val):
    color = 'red' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
df_document_topics

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9,Topic10,Topic11,Topic12,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,Topic19,dominant_topic
Doc0,0,0,0,0,0,0.98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
Doc1,0,0,0.02,0.08,0,0,0.05,0.03,0.23,0,0.23,0,0,0.33,0,0,0,0,0.03,0,13
Doc2,0,0,0.08,0.05,0,0,0.38,0,0,0,0,0.03,0,0,0,0,0.01,0,0.28,0.18,6
Doc3,0,0,0,0.34,0.11,0,0,0,0,0,0,0.02,0,0.4,0,0,0,0.06,0.07,0,13
Doc4,0,0,0,0.34,0.06,0,0.03,0,0.09,0,0.28,0,0.1,0.09,0,0,0,0,0,0,3
Doc5,0,0.03,0.27,0.11,0.15,0,0.1,0,0,0,0,0,0.11,0,0,0,0.07,0,0.15,0,2
Doc6,0.02,0,0,0,0,0.98,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
Doc7,0,0,0.22,0.5,0.05,0,0,0,0,0,0,0,0,0,0,0,0,0.22,0,0,3
Doc8,0,0.14,0,0,0,0,0,0,0,0,0.04,0,0,0.77,0,0,0,0,0.04,0,13
Doc9,0,0,0.04,0,0,0,0,0,0,0,0.06,0,0,0.68,0,0,0,0,0.21,0,13


In [64]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

/home/maza/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
18    -10.007368   9.912104       1        1  13.808583
14      2.103104  18.970848       2        1   9.399045
6      -6.903522  14.185094       3        1   8.731810
10     -4.819774  16.115082       4        1   7.536911
5       6.634317  20.317268       5        1   6.992977
3      -6.535084   3.391381       6        1   6.505084
7       5.627255  24.167942       7        1   6.061640
13      1.210410  28.604818       8        1   5.573945
0      -0.730429   1.278773       9        1   5.551811
19      5.159273   5.791086      10        1   4.655374
15     14.612535  23.373020      11        1   4.397605
2      16.080204  18.173489      12        1   4.093583
17      6.986467  17.946833      13        1   3.051003
4       8.022402  10.848022      14        1   2.544534
1       3.375220  25.966932      15        1   2.190014
11     12.620817   7.708573      16        1   2.161894
16     14.444958  12.683951      17        1   2.155434
12     -1.532498  33.750160      18        1   1.855726
8     -13.417603  20.841024      19        1   1.547276
9      -2.751891   7.671724      20        1   1.185753, topic_info=       Category           Freq  \
88592   Default  155941.000000   
55122   Default  242880.000000   
78605   Default  100327.000000   
3050    Default  256560.000000   
10280   Default  212917.000000   
48994   Default  113288.000000   
31217   Default  224979.000000   
70089   Default   23665.000000   
3036    Default   85787.000000   
73610   Default   43474.000000   
103285  Default   21233.000000   
95740   Default   45785.000000   
85241   Default   41334.000000   
44754   Default   34120.000000   
101997  Default   18808.000000   
88574   Default  152675.000000   
78673   Default   31518.000000   
80997   Default   29043.000000   
23765   Default   93831.000000   
3028    Default   57339.000000   
81574   Default   27147.000000   
91849   Default   50129.000000   
50571   Default   23316.000000   
38639   Default   54898.000000   
84842   Default   26509.000000   
66284   Default   24866.000000   
62522   Default   57389.000000   
25219   Default   27712.000000   
88422   Default   14747.000000   
78672   Default   25915.000000   
...         ...            ...   
8330    Topic20     469.358554   
65688   Topic20    3206.795394   
83358   Topic20    2995.494799   
22284   Topic20     681.363614   
75020   Topic20    6858.090592   
104398  Topic20    7573.079921   
11297   Topic20     936.560489   
57364   Topic20    3235.895302   
2933    Topic20    9094.860699   
9288    Topic20    1271.807495   
67455   Topic20    1633.458033   
4301    Topic20    1703.426327   
7413    Topic20    2567.425553   
67577   Topic20    5069.978606   
95990   Topic20    1418.725707   
41422   Topic20    2100.757959   
90005   Topic20    2272.660432   
75621   Topic20    2527.371655   
73610   Topic20    7673.596564   
57857   Topic20    2033.599555   
98009   Topic20    3807.326461   
103746  Topic20    5013.646664   
25219   Topic20    5359.992802   
70956   Topic20    3642.436263   
102331  Topic20    2123.829589   
104631  Topic20    3623.028304   
48994   Topic20    4679.139256   
3050    Topic20    5086.209149   
62522   Topic20    2732.559267   
48147   Topic20    2025.091092   

                                                     Term          Total  \
88592                                                 png  155941.000000   
55122                                           alternate  242880.000000   
78605                                                 jpg  100327.000000   
3050                                                 2019  256560.000000   
10280                                              BUTTON  212917.000000   
48994                                                 The  113288.000000   
31217                                                 Les  224979.000000   
70089   